In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, CovarianceStructure, ObjectiveLib
#import matplotlib.pyplot as plt
import pickle 

## Data Process

In [2]:
Nt = 8

max_manual_num = 10
min_interval_num = 10

In [3]:
print(np.linspace(0,60,9))

[ 0.   7.5 15.  22.5 30.  37.5 45.  52.5 60. ]


### API

In [4]:

static_ind = [0,1,2]
#dynamic_ind = [3,4,5]
dynamic_ind = [3]
all_ind = static_ind+dynamic_ind

num_total = len(all_ind)

In [5]:
#all_names_strategy3 = ["CA.static", "CB.static", "CC.static", 
#                      "CA.dynamic", "CB.dynamic", "CC.dynamic"]
all_names_strategy3 = ["CA.static", "CB.static", "CC.static", "CA.dynamic"]

In [6]:
#static_cost = [2000, # CA
#    2000, # CB
#     2000, # CC
#    200, # CA
#    200, # CB
#     200] # CC

static_cost = [2000, 2000, 2000, 200]
dynamic_cost = [0, 0, 0]
dynamic_cost.extend([400]*len(dynamic_ind))
#dynamic_cost = [0]*len(static_ind)
#dynamic_cost.extend([400]*len(dynamic_ind))

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

error_cov = [[1, 0.1, 0.1, 1], 
            [0.1, 4, 0.5, 0.1], 
            [0.1, 0.5, 8, 0.1], 
            [1, 0.1, 0.1, 1]]

#error_cov = [[1, 0.1, 0.1, 1, 0.1, 0.1],
#[0.1, 4, 0.5, 0.1, 4, 0.5],
#[0.1, 0.5, 8, 0.1, 0.5, 8], 
#[1, 0.1, 0.1, 1, 0.1, 0.1], 
#[0.1, 4, 0.5, 0.1, 4, 0.5], 
#[0.1, 0.5, 8, 0.1, 0.5, 8]]
'''

error_cov = [[1, 0.1, 0.1, 0, 0, 0],
[0.1, 4, 0.5, 0, 0, 0],
[0.1, 0.5, 8, 0, 0, 0], 
[0, 0, 0, 1, 0.1, 0.1], 
[0, 0, 0, 0.1, 4, 0.5], 
[0, 0, 0, 0.1, 0.5, 8]]
'''

'\n\nerror_cov = [[1, 0.1, 0.1, 0, 0, 0],\n[0.1, 4, 0.5, 0, 0, 0],\n[0.1, 0.5, 8, 0, 0, 0], \n[0, 0, 0, 1, 0.1, 0.1], \n[0, 0, 0, 0.1, 4, 0.5], \n[0, 0, 0, 0.1, 0.5, 8]]\n'

In [7]:
measure_info = pd.DataFrame({
    "name": all_names_strategy3,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

         name  Q_index  static_cost  dynamic_cost  min_time_interval  \
0   CA.static        0         2000             0                 10   
1   CB.static        1         2000             0                 10   
2   CC.static        2         2000             0                 10   
3  CA.dynamic        3          200           400                 10   

   max_manual_number  
0                 10  
1                 10  
2                 10  
3                 10  


In [8]:
dataObject = DataProcess()
dataObject.read_jacobian('./kinetics_fim/Q_drop0.csv')
#Q = dataObject.get_Q_list([0,1,2], [0,1,2], Nt)
Q = dataObject.get_Q_list([0,1,2], [0], Nt)


In [9]:
calculator = MeasurementOptimizer(Q, measure_info, error_cov=error_cov, error_opt=CovarianceStructure.measure_correlation, verbose=True)


fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

Covariances are between measurements at the same time.
(121, 4, 4)


## Solve

In [10]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt
#num_total = num_static

In [11]:
with open('./kinetics_results/May2_3000_a', 'rb') as f:
    init_cov_y = pickle.load(f)
#print(init_cov_y)

In [12]:
with open('./kinetics_results/May2_fim_3000_a', 'rb') as f:
    fim_prior = pickle.load(f)
    print(fim_prior)

[[  4.2421363    2.0974022   -7.15679971  -8.94930454]
 [  2.0974022    5.18300713  -2.8999988  -21.02439003]
 [ -7.15679971  -2.8999988   12.34045591  12.5645852 ]
 [ -8.94930454 -21.02439003  12.5645852   86.54108698]]


In [37]:
mip_option = True
objective = ObjectiveLib.D
sparse_opt = True
fix_opt = False

manual_num = 10
budget_opt = 2000

total_manual_init = 0
dynamic_install_init = [0,0,0] 

num_dynamic_time = np.linspace(0,60,9)

#static_dynamic = [[0,3],[1,4],[2,5]]
static_dynamic = [[0,3]]
time_interval_for_all = True

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time[1:]):
    dynamic_time_dict[i] = np.round(tim, decimals=2)
    
print(dynamic_time_dict)

{0: 7.5, 1: 15.0, 2: 22.5, 3: 30.0, 4: 37.5, 5: 45.0, 6: 52.5, 7: 60.0}


In [38]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    upper_diagonal_only=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    #init_cov_y= init_cov_y,
                    initial_fim = fim_prior,
                    dynamic_install_initial = dynamic_install_init, 
                    static_dynamic_pair=static_dynamic,
                    time_interval_all_dynamic = time_interval_for_all,
                    total_manual_num_init=total_manual_init)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

Starting MindtPy version 0.1.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7ff48722f820>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7ff48722f7f0>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x7ff48722fa30>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: true
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: false
load_solutions: true
heuristic_nonconvex: false
init_strategy: rNLP
level_coef: 0.5
soluti

3
[0, 0, 0]
3
[0, 0, 0]
initialized
initialized
initialized
initialized
initialized
initialized
initialized
initialized
initialized
initialized
solving rnlp
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
iteration
WARNING (W1002): Setting Var 'cov_y[0,10]' to a numeric value
`-2.1233322951504186e-09` outside the bounds (0, 1).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'cov_y[0,9]' to a numeric value
`-9.682534338534809e-10` outside the bounds (0, 1).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'cov_y[1,10]' to a numeric value
`-5.057934096021681e-09` outside the bounds (0, 1).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'cov_y[1,8]' to a numeric value
`-3.6034803835796676e-09` outside the bounds (0, 1

         -       Relaxed NLP          -2.68829           -inf       -2.68829      nan%      0.55


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-01
skipping further warning of this kind
Read LP format model from file /tmp/tmpx5ydxp8q.pyomo.lp
Reading time = 0.00 seconds
x1: 210 rows, 90 columns, 721 nonzeros
Read MIP start from file /tmp/tmp6tl7juns.gurobi.mst
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 210 rows, 90 columns and 721 nonzeros
Model fingerprint: 0x832d33d8
Variable types: 23 continuous, 67 integer (67 binary)
Coefficient statistics:
  Matrix range     [1e-13, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP start did not produce a new incumbent solution
Use

         1              MILP          -2.68829           -inf       -2.68829      nan%      0.71
*        1         Fixed NLP          -8.60721       -8.60721       -2.68829    68.77%      0.84


iteration
iteration
iteration
iteration
iteration
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-01
skipping further warning of this kind
Read LP format model from file /tmp/tmpzau1xpzh.pyomo.lp
Reading time = 0.00 seconds
x1: 211 rows, 90 columns, 732 nonzeros
Read MIP start from file /tmp/tmphkws4bck.gurobi.mst
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 211 rows, 90 columns and 732 nonzeros
Model fingerprint: 0x9514d508
Variable types: 23 continuous, 67 integer (67 binary)
Coefficient statistics:
  Matrix range     [1e-13, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP 

         2              MILP          -2.68829       -8.60721       -2.68829    68.77%      0.98
Cycling happens after 2 main iterations. The same combination is obtained in iteration 1 This issue happens when the NLP subproblem violates constraint qualification. Convergence to optimal solution is not guaranteed.
Final bound values: Primal Bound: -8.607207175818886  Dual Bound: -2.6882863113205047
 Primal integral          :    0.0000 
 Dual integral            :    0.0000 
 Primal-dual gap integral :    0.0000 


In [39]:
fim_result = np.zeros((4,4))
for i in range(4):
    for j in range(i,4):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

#print("Pyomo OF:", pyo.value(mod.Obj))
#print("Log_det:", np.log(np.linalg.det(fim_result)))

ans_y, sol_y = calculator.extract_solutions(mod)
print('pyomo calculated cost:', pyo.value(mod.cost))
print("if install dynamic measurements:")
print(pyo.value(mod.if_install_dynamic[3]))
#print(pyo.value(mod.if_install_dynamic[4]))
#print(pyo.value(mod.if_install_dynamic[5]))

[[  0.75317314   1.53825603  -1.0790614   -6.59330186]
 [  1.53825603   3.88725535  -2.1115524  -15.76829252]
 [ -1.0790614   -2.1115524    1.58585603   9.20472656]
 [ -6.59330186 -15.76829252   9.20472656  64.90581524]]
trace: 71.1320997493435
det: 0.00018278368069989418
[7.07138164e+01 4.09607776e-01 7.87415193e-03 8.01421814e-04]
CA.static :  0.0
CB.static :  1.0
CC.static :  0.0
CA.dynamic
[0. 0. 0. 0. 0. 0. 0. 0.]
pyomo calculated cost: 2000.0
if install dynamic measurements:
0.0
